# Data Visualization

This notebook demonstrates the process of visualizing data stored in Delta Lake using Dash

In [ ]:

from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

signal_name = "sensor_data_depth"
df = duckpond.conn.execute(f"""
SELECT signal_name, datetime, data_labels, values
FROM DataLake
WHERE signal_name = $1
""", [signal_name]).df()
display(df)

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import importlib

import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond()

labels = duckpond.conn.sql(f"""
    SELECT DISTINCT label
    FROM DataLake
    WHERE animal = 'oror-002'
    """).df()

labels = sorted([name for name in labels.label.to_list() if (name.startswith("sensor") or name.startswith("logger"))])

app = Dash()


fig = go.Figure()
dff = duckpond.get_delta_data(    
    animal_ids="oror-002",
    frequency=10,
    labels=labels,
)

# print(dff)
figures = {
    col: go.Figure()
    for col in dff.columns[1:3]
}

for idx, (col, fig) in enumerate(figures.items()):
    fig.add_trace(
        go.Scattergl(
            x=dff['datetime'],
            y=dff[col],
            mode='lines',
            name=col,
            yaxis=f'y{idx+1}'
        )
    )

# fig.update_layout(
#     yaxis=dict(title=dff.columns[1]),
#     yaxis2=dict(title=dff.columns[2])
# )

app.layout = [
    # dcc.Dropdown(labels, labels[0], id='dropdown-selection'),    
    dcc.Graph(id=f'graph-content-{idx}', figure=fig) for idx, fig in enumerate(figures.values())
]

app.run(debug=True, host='0.0.0.0')

In [ ]:
import dash
from dash import dcc, html, callback, Output, Input, State
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import importlib

# import DiveDB.services.duck_pond
# importlib.reload(DiveDB.services.duck_pond)
from DiveDB.DiveDB.services.duck_pond import DuckPond
from threejs_model import threejs_model

duckpond = DuckPond()

labels = duckpond.conn.sql(f"""
    SELECT DISTINCT label
    FROM DataLake
    WHERE animal = 'oror-002'
    """).df()

labels = sorted([name for name in labels.label.to_list() if (name.startswith("sensor") or name.startswith("logger"))])

app = dash.Dash(__name__)

dff = duckpond.get_delta_data(
    animal_ids="oror-002",
    frequency=1,
    labels=labels,
)

# Convert datetime to timestamp (seconds since epoch) for slider control
dff['timestamp'] = dff['datetime'].apply(lambda x: x.timestamp())

# Create the figure with subplots
fig = make_subplots(rows=len(dff.columns[1:-1]), cols=1, shared_xaxes=True, vertical_spacing=0.03)
for idx, col in enumerate(dff.columns[1:-1]):  # Exclude 'timestamp' column
    fig.add_trace(go.Scattergl(
        x=dff['datetime'],
        y=dff[col],
        mode='lines',
        name=col,
        showlegend=True
    ), row=idx+1, col=1)

# Set x-axis range to data range and set uirevision
fig.update_layout(
    xaxis=dict(
        range=[dff['datetime'].min(), dff['datetime'].max()]
    ),
    uirevision='constant'  # Maintain UI state across updates
)

#   fig.update_layout(
#         title='Tag Data Visualization',
#         xaxis=dict(
#             rangeselector=dict(
#                 buttons=[
#                     dict(count=30, label="30s", step="second", stepmode="backward"),
#                     dict(count=5, label="5m", step="minute", stepmode="backward"),
#                     dict(count=10, label="10m", step="minute", stepmode="backward"),
#                     dict(count=30, label="30m", step="minute", stepmode="backward"),
#                     dict(count=1, label="1h", step="hour", stepmode="backward"),
#                     dict(count=12, label="12h", step="hour", stepmode="backward"),
#                     dict(step="all", label="All")
#                 ]
#             ),
#             rangeslider=dict(
#                 visible=True,
#                 thickness=0.15
#             ),
#             type="date"
#         ),
#         height=600 + 50 * (len(signals_sorted) + extra_rows),
#         showlegend=True
#     )

# Define the app layout
app.layout = html.Div([
    dcc.Graph(id='graph-content', figure=fig),
    html.Div([
        html.Button('Play', id='play-button', n_clicks=0),
        dcc.Slider(
            id='playhead-slider',
            min=dff['timestamp'].min(),
            max=dff['timestamp'].max(),
            value=dff['timestamp'].min(),
            marks=None,
            tooltip={"placement": "bottom"}
        ),
        dcc.Interval(
            id='interval-component',
            interval=1*1000,  # Base interval of 1 second
            n_intervals=0,
            disabled=True  # Start with the interval disabled
        ),
        dcc.Store(id='playhead-time', data=dff['timestamp'].min()),
        dcc.Store(id='is-playing', data=False),
        threejs_model.ThreejsModel(
            id='input',
            value='my-value',
            label='my-label'
        ),
        html.Div(id='output')
    ])
])

@callback(Output('output', 'children'), Input('input', 'value'))
def display_output(value):
    return 'You have entered {}'.format(value)

# Callback to toggle play/pause state
@app.callback(
    Output('is-playing', 'data'),
    Output('play-button', 'children'),
    Input('play-button', 'n_clicks'),
    State('is-playing', 'data')
)
def play_pause(n_clicks, is_playing):
    if n_clicks % 2 == 1:
        return True, 'Pause'  # Switch to playing
    else:
        return False, 'Play'  # Switch to paused

# Callback to enable/disable the interval component based on play state
@app.callback(
    Output('interval-component', 'disabled'),
    Input('is-playing', 'data')
)
def update_interval_component(is_playing):
    return not is_playing  # Interval is disabled when not playing

# Callback to update playhead time based on interval or slider input
@app.callback(
    Output('playhead-time', 'data'),
    Output('playhead-slider', 'value'),
    Input('interval-component', 'n_intervals'),
    Input('playhead-slider', 'value'),
    State('is-playing', 'data'),
    prevent_initial_call=True
)
def update_playhead(n_intervals, slider_value, is_playing):
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate

    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if trigger_id == 'interval-component' and is_playing:
        # Find the current index based on the slider value
        current_idx = dff['timestamp'].sub(slider_value).abs().idxmin()
        next_idx = current_idx + 5 if current_idx + 5 < len(dff) else 0  # Loop back to start
        new_time = dff['timestamp'].iloc[next_idx]
        return new_time, new_time
    elif trigger_id == 'playhead-slider':
        return slider_value, slider_value
    else:
        raise dash.exceptions.PreventUpdate

# Callback to update the graph with the playhead line
@app.callback(
    Output('graph-content', 'figure'),
    Input('playhead-time', 'data'),
    State('graph-content', 'figure')
)
def update_graph(playhead_timestamp, existing_fig):
    playhead_time = pd.to_datetime(playhead_timestamp, unit='s')
    existing_fig['layout']['shapes'] = []
    existing_fig['layout']['shapes'].append(
        dict(
            type='line',
            x0=playhead_time,
            x1=playhead_time,
            y0=0,
            y1=1,
            xref='x',
            yref='paper',
            line=dict(
                width=2,
                dash='solid'
            )
        )
    )
    existing_fig['layout']['uirevision'] = 'constant'
    return existing_fig

if __name__ == '__main__':
    app.run_server(debug=True, host='0.0.0.0', port=8052)

In [ ]:
from threejs_model import threejs_model
import os

dirs = os.listdir(
    os.path.join(
        os.getcwd(),
        "threejs_model"
    )
)
for i in dirs:
    print(i)
# help(threejs_model)

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go

app = dash.Dash(__name__)

# Example data
x = [i for i in range(100)]
y1 = [i**2 for i in x]
y2 = [i**0.5 for i in x]

# Layout for the graph with initial settings
def create_graph(id, x, y):
    return dcc.Graph(
        id=id,
        figure=go.Figure(
            data=[go.Scatter(x=x, y=y, mode='lines')],
            layout=go.Layout(
                xaxis={'range': [0, 100]},
                yaxis={'range': [0, 100]},
                margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                height=300,
            )
        )
    )

app.layout = html.Div([
    create_graph('graph-1', x, y1),
    create_graph('graph-2', x, y2),
])

# Callback to sync the x and y axis ranges across graphs
@app.callback(
    [Output('graph-2', 'figure'),
     Output('graph-1', 'figure')],
    [Input('graph-1', 'relayoutData'),
     Input('graph-2', 'relayoutData')],
    [State('graph-1', 'figure'),
     State('graph-2', 'figure')]
)
def sync_viewport(graph1_layout, graph2_layout, fig1, fig2):
    ctx = dash.callback_context
    triggered_graph = ctx.triggered[0]['prop_id'].split('.')[0]

    if triggered_graph == 'graph-1' and graph1_layout is not None:
        if 'xaxis.range[0]' in graph1_layout and 'xaxis.range[1]' in graph1_layout:
            fig2['layout']['xaxis']['range'] = [graph1_layout['xaxis.range[0]'], graph1_layout['xaxis.range[1]']]
        if 'yaxis.range[0]' in graph1_layout and 'yaxis.range[1]' in graph1_layout:
            fig2['layout']['yaxis']['range'] = [graph1_layout['yaxis.range[0]'], graph1_layout['yaxis.range[1]']]
        return fig2, fig1
    elif triggered_graph == 'graph-2' and graph2_layout is not None:
        if 'xaxis.range[0]' in graph2_layout and 'xaxis.range[1]' in graph2_layout:
            fig1['layout']['xaxis']['range'] = [graph2_layout['xaxis.range[0]'], graph2_layout['xaxis.range[1]']]
        if 'yaxis.range[0]' in graph2_layout and 'yaxis.range[1]' in graph2_layout:
            fig1['layout']['yaxis']['range'] = [graph2_layout['yaxis.range[0]'], graph2_layout['yaxis.range[1]']]
        return fig2, fig1
    return fig2, fig1

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=8051)

